### Librerias

In [0]:
spark.conf.set("spark.databricks.io.cache.enabled", True)
spark.conf.set('spark.sql.shuffle.partitions', 'auto')

In [0]:
%run ../../../../../04_utils/commons_functions_de

### Librerías

### Funciones Ingenieria de datos

### Funciones de ingesta en RDS

### Funciones de control de flujo de ingesta

In [0]:
%run ../../../../../04_utils/commons_functions_ds

In [0]:
%run ../../../../../spigot/initial/global_parameter_py

In [0]:
from pyspark.sql.types import IntegerType
import pyspark.sql.functions as F
import pandas as pd
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number
from pyspark.sql.functions import regexp_extract

com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:447)
	at com.databricks.spark.chauffeur.ChauffeurState.cancelExecution(ChauffeurState.scala:1315)
	at com.databricks.spark.chauffeur.ChauffeurState.$anonfun$process$1(ChauffeurState.scala:1032)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperation$1(UsageLogging.scala:573)
	at com.databricks.logging.UsageLogging.executeThunkAndCaptureResultTags$1(UsageLogging.scala:669)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperationWithResultTags$4(UsageLogging.scala:687)
	at com.databricks.logging.UsageLogging.$anonfun$withAttributionContext$1(UsageLogging.scala:426)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
	at com.databricks.logging.AttributionContext$.withValue(AttributionContext.scala:216)
	at com.databricks.logging.UsageLogging.withAttributionContext(UsageLogging.scala:424)
	at com.databricks.logging.Usa

#### Carga de Fuente

In [0]:
int_pedidos_clientes = (spark.read.parquet("/Volumes/dbw_prod_aavanzada/db_tmp/files/pburbano/data/")
                                  .withColumn("fecha_pedido_dt", F.to_date(F.col("fecha_pedido_dt")))
                        )

#### MDT

In [0]:
# Función de transformaciones iniciales
def transformar_variables_iniciales(df):
    df = df.withColumn("madurez_digital_val",
                    F.when(F.col("madurez_digital_cd") == "BAJA", 1)
                    .when(F.col("madurez_digital_cd") == "MEDIA", 2)
                    .when(F.col("madurez_digital_cd") == "ALTA", 3)
                    .otherwise(None)
                )

    df = df.withColumn("estrellas_val", F.col("estrellas_txt").cast("int"))
    df = df.withColumn("frecuencia_visitas_val", F.length(F.col("frecuencia_visitas_cd")))
    return df


# Función para definir ventanas 
def definir_ventanas():
    w = Window.partitionBy("cliente_id").orderBy(F.asc("fecha_pedido_dt"))
    w_prev_all = w.rowsBetween(Window.unboundedPreceding, -1)
    w_recent = w.rowsBetween(-3, -1)
    return w, w_prev_all, w_recent


# Función para crear target multiclase 
def crear_target(df, w):
    df = df.withColumn("canal_siguiente", F.lead("canal_pedido_cd").over(w))
    df = df.filter(F.col("canal_siguiente").isNotNull())

    df = df.withColumn("target",
                F.when((F.col("canal_pedido_cd") != "DIGITAL") & (F.col("canal_siguiente") != "DIGITAL"), 0)
                .when((F.col("canal_pedido_cd") != "DIGITAL") & (F.col("canal_siguiente") == "DIGITAL"), 1)
                .when((F.col("canal_pedido_cd") == "DIGITAL") & (F.col("canal_siguiente") == "DIGITAL"), 2)
                .when((F.col("canal_pedido_cd") == "DIGITAL") & (F.col("canal_siguiente") != "DIGITAL"), 3)
    )
    return df


# Función para variables de transiciones
def agregar_variables_canal(df, w):
    return (
        df.withColumn("canal_previo", F.lag("canal_pedido_cd").over(w))
          .withColumn("canal_actual", F.col("canal_pedido_cd"))
          .withColumn("canal_actual_digital", F.when(F.col("canal_actual") == "DIGITAL", 1).otherwise(0))
          .withColumn("canal_previo_digital", F.when(F.col("canal_previo") == "DIGITAL", 1).otherwise(0))
    )


# Función para variables históricas 
def agregar_variables_historicas(df, w, w_prev_all):
    return (
        df.withColumn("dias_desde_pedido_anterior", F.datediff("fecha_pedido_dt", F.lag("fecha_pedido_dt").over(w)))
          .withColumn("n_pedidos_previos", F.row_number().over(w) - 1)
          .withColumn("facturacion_prom_anterior", F.avg("facturacion_usd_val").over(w_prev_all))
          .withColumn("facturacion_total_prev", F.sum("facturacion_usd_val").over(w_prev_all))
          .withColumn("desviacion_facturacion", F.stddev("facturacion_usd_val").over(w_prev_all))
          .withColumn("uso_digital_prev", F.sum(F.when(F.col("canal_pedido_cd") == "DIGITAL", 1).otherwise(0)).over(w_prev_all))
          .withColumn("uso_no_digital_prev", F.col("n_pedidos_previos") - F.col("uso_digital_prev"))
          .withColumn("prop_digital_prev", 
                      F.when(F.col("n_pedidos_previos") > 0,
                             F.col("uso_digital_prev") / F.col("n_pedidos_previos")).otherwise(0))
          .withColumn("prop_no_digital_prev", 
                      F.when(F.col("n_pedidos_previos") > 0,
                             F.col("uso_no_digital_prev") / F.col("n_pedidos_previos")).otherwise(0))
          .withColumn("dias_media_prev", 
                      F.avg(F.datediff("fecha_pedido_dt", F.lag("fecha_pedido_dt").over(w))).over(w_prev_all))
          .withColumn("dias_media_std", 
                      F.stddev(F.datediff("fecha_pedido_dt", F.lag("fecha_pedido_dt").over(w))).over(w_prev_all))
    )


# Función para variables reciente
def agregar_variables_recientes(df, w_recent):
    return (
        df.withColumn("facturacion_prom_reciente", F.avg("facturacion_usd_val").over(w_recent))
          .withColumn("uso_digital_reciente", F.avg(F.when(F.col("canal_pedido_cd") == "DIGITAL", 1).otherwise(0)).over(w_recent))
          .withColumn("uso_no_digital_reciente", F.avg(F.when(F.col("canal_pedido_cd") != "DIGITAL", 1).otherwise(0)).over(w_recent))
    )


# Función para variables de materiales y cajas
def agregar_variables_materiales(df, w_prev_all, w_recent):
    return (
        df.withColumn("materiales_prom_prev", F.avg("materiales_distintos_val").over(w_prev_all))
          .withColumn("materiales_total_prev", F.sum("materiales_distintos_val").over(w_prev_all))
          .withColumn("cajas_fisicas_prom_prev", F.avg("cajas_fisicas").over(w_prev_all))
          .withColumn("cajas_fisicas_total_prev", F.sum("cajas_fisicas").over(w_prev_all))
          .withColumn("materiales_reciente", F.avg("materiales_distintos_val").over(w_recent))
          .withColumn("cajas_fisicas_reciente", F.avg("cajas_fisicas").over(w_recent))
          .withColumn("cajas_por_material", 
                      F.when(F.col("materiales_distintos_val") > 0, 
                             F.col("cajas_fisicas") / F.col("materiales_distintos_val"))
                       .otherwise(0))
          .withColumn("cajas_por_material_prev", 
                      F.avg(F.when(F.col("materiales_distintos_val") > 0, 
                                   F.col("cajas_fisicas") / F.col("materiales_distintos_val"))
                            .otherwise(0)).over(w_prev_all))
    )


# Función para variables temporales
def agregar_variables_temporales(df):
    return (
        df.withColumn("mes", F.month("fecha_pedido_dt"))
          .withColumn("dia_semana", F.dayofweek("fecha_pedido_dt"))
          .withColumn("es_fin_de_semana", F.when(F.col("dia_semana").isin(1, 7), 1).otherwise(0))
          .withColumn("trimestre", F.quarter("fecha_pedido_dt"))
    )


# Función para antigüedad del cliente
def agregar_antiguedad(df):
    return df.withColumn(
        "antiguedad_dias",
        F.datediff("fecha_pedido_dt", F.min("fecha_pedido_dt").over(Window.partitionBy("cliente_id")))
    )


# Función para selección final y periodos train y test
def seleccionar_variables_finales(df):
    fecha_corte = "2024-03-01"
    mdt = (
        df.filter(F.col("n_pedidos_previos") > 0)
          .filter(F.col("target").isNotNull())
          .select(
              "cliente_id", "pais_cd", "region_comercial_txt", "agencia_id", "ruta_id",
              "tipo_cliente_cd", "madurez_digital_cd", "estrellas_txt", "frecuencia_visitas_cd",
              "target",
              "canal_actual_digital", "canal_previo_digital",
              "facturacion_usd_val", "dias_desde_pedido_anterior", "n_pedidos_previos",
              "facturacion_prom_anterior", "facturacion_total_prev", "desviacion_facturacion",
              "uso_digital_prev", "uso_no_digital_prev", "prop_digital_prev", "prop_no_digital_prev",
              "facturacion_prom_reciente", "uso_digital_reciente", "uso_no_digital_reciente",
              "dias_media_prev", "dias_media_std",
              "materiales_distintos_val", "materiales_prom_prev", "materiales_total_prev", "materiales_reciente",
              "cajas_fisicas", "cajas_fisicas_prom_prev", "cajas_fisicas_total_prev", "cajas_fisicas_reciente",
              "cajas_por_material", "cajas_por_material_prev",
              "mes", "dia_semana", "es_fin_de_semana", "trimestre",
              "antiguedad_dias", "fecha_pedido_dt",
              "madurez_digital_val", "estrellas_val", "frecuencia_visitas_val"
          )
    )
    mdt = mdt.withColumn("periodo", F.when(F.col("fecha_pedido_dt") < fecha_corte, "TRAIN").otherwise("TEST"))
    return mdt.fillna(0)


# Función para unir proporciones de rua y agencia
def agregar_proporciones_sin_leakage(mdt):
    prop_agencia = (
        mdt.filter("periodo == 'TRAIN'")
           .groupBy("agencia_id")
           .agg(F.avg(F.when(F.col("target").isin(2, 3), 1).otherwise(0)).alias("prop_digital_agencia"))
    )
    prop_ruta = (
        mdt.filter("periodo == 'TRAIN'")
           .groupBy("ruta_id")
           .agg(F.avg(F.when(F.col("target").isin(2, 3), 1).otherwise(0)).alias("prop_digital_ruta"))
    )
    return (
        mdt.join(prop_agencia, on="agencia_id", how="left")
           .join(prop_ruta, on="ruta_id", how="left")
    )


# Función principal de construccion de la mdt
def construir_mdt(int_pedidos_clientes):
    df = transformar_variables_iniciales(int_pedidos_clientes)
    w, w_prev_all, w_recent = definir_ventanas()
    df = crear_target(df, w)
    df = agregar_variables_canal(df, w)
    df = agregar_variables_historicas(df, w, w_prev_all)
    df = agregar_variables_recientes(df, w_recent)
    df = agregar_variables_materiales(df, w_prev_all, w_recent)
    df = agregar_variables_temporales(df)
    df = agregar_antiguedad(df)
    mdt = seleccionar_variables_finales(df)
    mdt = agregar_proporciones_sin_leakage(mdt)
    return mdt


# Ejecución del pipeline
mdt = construir_mdt(int_pedidos_clientes)